In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
data = pd.read_csv('data/housing.csv')

In [8]:
data = pd.get_dummies(data)

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data.drop(['SalePrice'], axis=1), 
                                                    data.SalePrice, test_size=0.3, 
                                                    random_state=0)

# ExhaustiveSearch

它创建了从M到N的所有特征子集，其中N是特征的总数，并且对于每个子集，它构建一个机器学习算法并选择具有最佳性能的子集。

* scoring:我们评估模型性能的评分函数。请务必分别为回归和分类情况设置正确的设置。
* cv：K-fold 交叉验证的折叠数——如果cv为 0 或 False，则不进行交叉验证。

In [10]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector
           
# import the algorithm you want to evaluate on your features.
from sklearn.ensemble import RandomForestClassifier

# create the ExhaustiveFeatureSelector object.
efs = ExhaustiveFeatureSelector(RandomForestClassifier(), 
           min_features=4,
           max_features=10, 
           scoring='neg_mean_squared_error',
           cv=2)

In [11]:
# fit the object to the training data.
efs = efs.fit(x_train, y_train)

f:\Code\python\AILearn\env\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
f:\Code\python\AILearn\env\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
f:\Code\python\AILearn\env\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
f:\Code\python\AILearn\env\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
f:\Code\python\AILearn\env\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
f:\Code\python\AILea

In [ ]:
# print the selected features.
selected_features = x_train.columns[list(efs.k_feature_idx_)]
print(selected_features)

# print the final prediction score.
print(efs.k_score_)

# transform our data to the newly selected features.
x_train_sfs = efs.transform(x_train)
x_test_sfs = efs.transform(x_test)

# StepBackward & StepForward
前向搜索是将特征从0个开始不停增加特征，每次增加最有用的特征，一直增加到评分达到某个条件；后向搜索则相反，从全集开始删除特征，一直到评分达到某个条件。
## 限制
* 因为我们知道 SFS 在每次迭代中都会添加特征，所以当我们添加一个在开始时有用的特征，但在添加更多之后，现在没有用时，可能会出现问题。在这一点上，没有办法删除这种功能。
* 同样的事情发生在 SBS 上，但方向相反——这是因为 SBS 在从特征集中删除后无法看到特征的有用性。

## LRS & Sequential Floating
### LRS or Plus-L, minus-R
算法增加了两个参数L和R，如果 L>R，LRS 从空特征集开始，并且：
* 反复添加L个特征
* 反复删除R特征
如果 L < R，LRS 从完整的特征集开始，并且：
* 反复删除R特征
* 反复添加L个特征
LRS 试图通过一些回溯功能来弥补 SFS 和 SBS 的弱点，但是这种方法需要仔细设置 L 和 R 参数，而且这种选择最佳值背后缺乏理论。

### Sequential Floating
顺序浮动是 LRS 的扩展。我们不用选择 L 和 R 的值来添加和删除特征，而是直接从数据中确定这一点。
以SFFS(浮动前向步进选择)为例：
* 从一个空集开始。
* 像我们通常在 SFS 中所做的那样选择最佳功能并添加它。
* 然后从这个子集中选择最差的特征。
* 通过删除此特征来评估并检查目标函数是否有所改善。如果它正在改进，我们删除此功能；否则，我们保留它。
* 从第 2 步开始重复，直到达到停止标准。

## StepBackward
### SequentialFeatureSelector
* k_features:从 0 开始时要达到的最大特征——这是预设的标准。
* forward:使用此对象进行向前或向后的特征选择。
* floating:使用步进选择的一种变体，称为步进浮动向前选择——我们稍后会详细讨论。
* scoring:我们评估模型性能的评分函数。请务必分别为回归和分类情况设置正确的设置。
* cv：K-fold 交叉验证的折叠数——如果cv为 0 或 False，则不进行交叉验证。

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector

# import the algorithm you want to evaluate on your features.
from sklearn.ensemble import RandomForestRegressor

# just set forward=False for backward feature selection.
# create theSequentialFeatureSelector object, and configure the parameters.
sbs = SequentialFeatureSelector(RandomForestClassifier(), 
           k_features=10, 
           forward=False, 
           floating=False,
           scoring='accuracy',
           cv=2)

In [ ]:
# fit the object to our training data.
sbs = sbs.fit(x_train, y_train)

In [ ]:
# print the selected features. 
selected_features = x_train.columns[list(sbs.k_feature_idx_)]
print(selected_features)

# print the final prediction score.
print(sbs.k_score_)

# transform to the newly selected features.
x_train_sfs = sbs.transform(x_train)
x_test_sfs = sbs.transform(x_test)

## StepForward

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector

# import the algorithm you want to evaluate on your features.
from sklearn.ensemble import RandomForestRegressor

# create the SequentialFeatureSelector object, and configure the parameters.
sfs = SequentialFeatureSelector(RandomForestRegressor(), 
           k_features=10, 
           forward=True, 
           floating=False,
           scoring='accuracy',
           cv=2)

In [ ]:
# fit the object to the training data.
sfs = sfs.fit(x_train, y_train)

In [ ]:
# print the selected features.
selected_features = x_train.columns[list(sfs.k_feature_idx_)]
print(selected_features)

# print the final prediction score.
print(sfs.k_score_)

# transform to the newly selected features.
x_train_sfs = sfs.transform(x_train)
x_test_sfs = sfs.transform(x_test)

# 双向搜索（BSD)
BDS 同时应用 SFS 和 SBS，先做前向特征选择，然后每次做完一轮前向特征选择马上做后向特征选择，一直到结束。或者先做后向特征选择，然后做前向特征选择。为了防止死循环，需要做如下约束：
* SFS 已选择的特征不会被 SBS 删除。
* SBS 已删除的功能不会由 SFS 添加。
* 1次前向+1次后向为1轮，约束从第二轮开始生效